In [ ]:
import pywt 
import features
from matplotlib import pyplot as plt, rc_context
import numpy as np
import scipy
import add_labels
from mpl_styles import list_sizes

# Simulate signal

In [ ]:
sim_fs = 4096
sim_dt = 1/sim_fs
sim_x = np.arange(0, 4, sim_dt)  # time
sim_b = sp.signal.chirp(sim_x, 32, sim_x[-1], sim_fs//2, method='logarithmic')
sim_b += np.random.normal(0, .1, size=len(sim_x))

spectrum, _, _, im = plt.specgram(sim_b, Fs=sim_fs)
plt.yscale('symlog', base=2)
add_labels.spec()
# plt.colorbar(im)

plt.savefig(OUT_PATH / 'sim_b_spec.png')

# DTCWT

In [ ]:
plt.figure()
scalogram = features.dtcwt(sim_b, 13, False, True)
im = plt.imshow(scalogram)
add_labels.dwt()
# plt.colorbar(im)
plt.tight_layout()
plt.savefig(OUT_PATH / 'sim_b_dtcwt.png')

# DWT

In [ ]:
plt.figure(figsize=list_sizes['A4/2'])

plt.subplot(2, 2, 1)
im = plt.imshow(features.dwt(sim_b, 'db1', 13))
add_labels.dwt()

plt.subplot(2, 2, 2)
im = plt.imshow(features.dwt(sim_b, 'db8', 13))
add_labels.dwt()

plt.subplot(2, 2, 3)
im = plt.imshow(features.dwt(sim_b, 'bior1.5', 13))
add_labels.dwt()

plt.subplot(2, 2, 4)
im = plt.imshow(features.dwt(sim_b, 'dmey', 13))
add_labels.dwt()

plt.tight_layout()
plt.savefig(OUT_PATH / 'sim_b_dwt.png')

# MRA (SWT + DWT)

In [ ]:
plt.figure(figsize=list_sizes['A4/2'] / np.array([1, 2]))

plt.subplot(1, 2, 1)
cA, *cDs = pywt.mra(sim_b, 'db8', level=13, transform='swt')
arr = np.stack(reversed(cDs))
im = plt.imshow(arr)
add_labels.dwt()

plt.subplot(1, 2, 2)
cA, *cDs = pywt.mra(sim_b, 'db8', level=13, transform='dwt')
arr = np.stack(reversed(cDs))
im = plt.imshow(arr)
add_labels.dwt()

plt.tight_layout()
plt.savefig(OUT_PATH / 'sim_b_mra.png')

# Cont WT

In [ ]:
w = 6.
freq_num = 63
freqs = np.linspace(1, sim_fs/2, freq_num)
widths = w*sim_fs / (2*freqs*np.pi)
im = scipy.signal.cwt(sim_b, scipy.signal.morlet2, widths, w=w)
plt.imshow(np.flip(np.abs(im)))
plt.xlabel(add_labels.samples)
plt.ylabel(add_labels.hz)
pass

# All

In [ ]:
plt.figure(figsize=list_sizes['A4'])
def subplot(n):
    plt.subplot(4, 2, n)

with rc_context({'axes.grid': False}):
    subplot(1)
    plt.title('Оконное преобразование Фурье')
    spectrum, _, _, im = plt.specgram(sim_b, Fs=sim_fs)
    plt.yscale('symlog', base=2)
    add_labels.spec()
    subplot(2)
    plt.title('КВП-ДД (13 ур.)')
    plt.imshow(features.dtcwt(sim_b, 13, False, True))
    add_labels.dwt()
    subplot(3)
    plt.title('ДВП (db1, 13 ур.)')
    im = plt.imshow(features.dwt(sim_b, 'db1', 13))
    add_labels.dwt()
    subplot(4)
    plt.title('ДВП (db8, 13 ур.)')
    im = plt.imshow(features.dwt(sim_b, 'db8', 13))
    add_labels.dwt()
    subplot(5)
    plt.title('ДВП (bior1.5, 13 ур.)')
    im = plt.imshow(features.dwt(sim_b, 'bior1.5', 13))
    add_labels.dwt()
    subplot(6)
    plt.title('ДВП (dmey, 13 ур.)')
    im = plt.imshow(features.dwt(sim_b, 'dmey', 13))

    subplot(7)
    plt.title('MRA ДВП (db8, 13 ур.)')
    cA, *cDs = pywt.mra(sim_b, 'db8', level=13, transform='dwt')
    arr = np.stack(reversed(cDs))
    im = plt.imshow(arr)
    add_labels.dwt()

    subplot(8)
    plt.title('MRA СВП (db8, 13 ур.)')
    cA, *cDs = pywt.mra(sim_b, 'db8', level=13, transform='swt')
    arr = np.stack(reversed(cDs))
    im = plt.imshow(arr)
    add_labels.dwt()

    plt.tight_layout()
    plt.savefig(OUT_PATH / 'sim_b_all.png')